# Subsidence

Notebook to migrate xlsx files to CF compliant ..

In [1]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

<IPython.core.display.Javascript object>

### Configure OS dependent paths

In [2]:
# Import standard packages
import os
import pathlib

import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import json
import copy
from itertools import chain
from shapely import wkb

# Import custom functionality
from coclicodata.drive_config import p_drive
from coclicodata.etl.cf_compliancy_checker import check_compliancy, save_compliancy

# Define (local and) remote drives
gca_data_dir = p_drive.joinpath("11208003-latedeo2022","020_InternationalDeltaPortfolio","datasets")
coclico_data_dir = p_drive.joinpath("11205479-coclico", "FASTTRACK_DATA")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    pathlib.Path().home().joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"Anaconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml"
    )
)

C:\Users\kras\AppData\Local\Temp\ipykernel_9920\3046747413.py:6: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


<IPython.core.display.Javascript object>

In [3]:
# Project paths & files (manual input)
dataset_dir = gca_data_dir.joinpath(r"00_mapping_global_threat_of_land_subsidence")
dataset_dir_path = dataset_dir.joinpath("abb8549_data_s3_red.nc")
dataset_out_file = "Global_TLS" # threat of land subsidence
CF_dir = gca_data_dir.joinpath(r"CF")  # directory to save output CF check files

<IPython.core.display.Javascript object>

### Write XLSX to NetCDF

In [4]:
# write xlsx to netcdf

# open the XLSX dataset as pandas dataframe
df = pd.read_excel(str(dataset_dir_path).replace("_red.nc", ".xlsx"), index_col=None, header=0)

# Rename columns that cause errors
key_list_corr = {}
for idx, i in enumerate(df.keys()):
    if '/' in i:
        key_list_corr[i] = i.replace("/", "per")

df = df.rename(columns=key_list_corr)

# Select relevant columns (info from Gilles --> decreases the datasets)
columns=['Country',
        'Potential exposed population in 2010 (Million)',
        'Exposed GDP (EGDP) (Billion US$)',
        'Potential global subsidence index in 2010 (PGSI)',
        'Potential exposed population in 2040 (Million)',
        'Exposed GDP (EGDP) (Billion US$) in 2040',
        'Potential subsidence index 2040',
        ]

df = df[columns]

# Convert the pandas dataframe to an xarray dataset
ds = xr.Dataset.from_dataframe(df)

# # Write the xarray dataset to a netCDF file
ds.to_netcdf(dataset_dir_path)

<IPython.core.display.Javascript object>

### Check CF compliancy original NetCDF files

In [5]:
# open datasets
ds = xr.open_dataset(dataset_dir_path)

# check original dataset
ds

<xarray.Dataset>
Dimensions:                                           (index: 110)
Coordinates:
  * index                                             (index) int64 0 1 ... 109
Data variables:
    Country                                           (index) object ...
    Potential exposed population in 2010 (Million)    (index) float64 ...
    Exposed GDP (EGDP) (Billion US$)                  (index) float64 ...
    Potential global subsidence index in 2010 (PGSI)  (index) float64 ...
    Potential exposed population in 2040 (Million)    (index) float64 ...
    Exposed GDP (EGDP) (Billion US$) in 2040          (index) float64 ...
    Potential subsidence index 2040                   (index) float64 ...

<IPython.core.display.Javascript object>

In [6]:
%%capture cap --no-stderr
# check original CF compliancy

check_compliancy(testfile=dataset_dir_path, 
                 working_dir=CF_dir
                 )

<IPython.core.display.Javascript object>

In [7]:
# save original CF compliancy
save_compliancy(cap, testfile=dataset_dir_path, working_dir=CF_dir)

{'P:\\11208003-latedeo2022\\020_InternationalDeltaPortfolio\\datasets\\00_mapping_global_threat_of_land_subsidence\\abb8549_data_s3_red.nc': {'warnings': '17', 'errors': '1'}}


<IPython.core.display.Javascript object>

### Add NUTS regions (or equivalent)

The nuts regions are not included as attributes in the netCDF files. Also this dataset is global and hence NUTS cannot be used. The NetCDF files only contain country names so we retrieve that information the [World Bank](https://datacatalog.worldbank.org/search/dataset/0038272/World-Bank-Official-Boundaries).

In [8]:
# load country regions
glob_regions = gpd.read_file(
    coclico_data_dir.joinpath("XX_Global", "WB_countries_Admin0.geojson")
)
glob_regions = glob_regions.to_crs("EPSG:4326")

glob_regions["polygons"] = glob_regions.geometry  # rename the geometry
glob_regions["geometry"] = glob_regions.centroid.buffer(0.5)  # set the buffered centroid as geometry (for simplicity)
glob_regions["lon"] = glob_regions.centroid.x # set the lon
glob_regions["lat"] = glob_regions.centroid.y # set the lat
#glob_regions.representative_point() # usefull if for instance Aruba was part of NL

# plot 
# %matplotlib ipympl
# %matplotlib inline

# fig, ax = plt.subplots(figsize=(14,7))
# glob_regions["polygons"].plot(ax=ax) # all countries that could be plotted
# glob_regions.centroid.plot(ax=ax, markersize=5, color="red")
# glob_regions.representative_point().plot(ax=ax, markersize=5, color="green")

# plt.grid(alpha=0.5)

C:\Users\kras\AppData\Local\Temp\ipykernel_9920\1845176697.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  glob_regions["geometry"] = glob_regions.centroid.buffer(0.5)  # set the buffered centroid as geometry (for simplicity)
C:\Users\kras\AppData\Local\Temp\ipykernel_9920\1845176697.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  glob_regions["geometry"] = glob_regions.centroid.buffer(0.5)  # set the buffered centroid as geometry (for simplicity)
C:\Users\kras\AppData\Local\Temp\ipykernel_9920\1845176697.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  glob_reg

<IPython.core.display.Javascript object>

In [9]:
ds_old = ["China", "United States", "Taiwan", "Palestina", "Macedonia", "Caspian Sea", "Republic of Congo"] # non WB compliant country names
ds_new = ["People's Republic of China", "United States of America", "", "Palestine", "Republic of Macedonia", "", "Republic of the Congo"] # WB country name

# replace countries with WB compliant country names
idx_rep = [idx for idx, x in enumerate(ds['Country'].values) if x in ds_old] # get ids for replacement task
ds['Country'].values[idx_rep] = ds_new

<IPython.core.display.Javascript object>

In [10]:
# join datasets by keeping correct countries

# check = []
# for i in ds["Country"].values:
#     if i in list(glob_regions["NAME_EN"]):
#         check.append(i)
#         #print("in", i)
#     else:
#         print("not in", i)

# make xarray dataframe
dspd = ds.to_dataframe()

# note, this automatically removes country = "" from the dataframe (Taiwan; is part of China, and Caspian Sea)
dsgr = dspd.merge(glob_regions, left_on='Country', right_on='NAME_EN')

# note, duplications in NL and NZ, we will only keep the biggest area polygons (the actual countries)
def dup_idx(lst, item):
    return [i for i, x in enumerate(lst) if x == item]

rem_list = []
for i in np.unique(dsgr.Country):
    idx_dupl = dup_idx(dsgr.Country, i) # check if duplicate items in the list
    if len(idx_dupl) > 1: # only retrieve area when more than one
        area_dupl = [dsgr.Shape_Area[j] for j in idx_dupl] # get area of duplicate items
        idx_dupl.pop(np.argmax(area_dupl)) # remove the one with the largest area
        rem_list.append(idx_dupl)

dsgr_red = dsgr.drop(list(chain.from_iterable(rem_list))) # reduce dataframe with duplicate idx  

# plot 
# %matplotlib ipympl
# %matplotlib inline

# fig, ax = plt.subplots(figsize=(14,7))
# gdf = gpd.GeoDataFrame(dsgr_red, geometry="polygons")
# gdf.plot(ax=ax) # reduced countries with data
# plt.scatter(gdf.lon, gdf.lat, s=5, color="red")
# plt.grid(alpha=0.5)

<IPython.core.display.Javascript object>

In [11]:
# drop countries with "" from the dataframe
idx_drop = [idx for idx, i in enumerate(ds.Country) if i == ""] 
ds = ds.drop_sel(index=idx_drop)

<IPython.core.display.Javascript object>

In [12]:
# add geometries and lon, lat to file

# extract geometries of nut2 regions in well-known binary format
geoms = dsgr_red["geometry"].apply(lambda x: wkb.dumps(x))

# rename dims and add new data to dataset
ds = ds.assign_coords({"geometry": ("index", geoms)})
ds = ds.assign_coords(lon=("index", dsgr_red.lon))
ds = ds.assign_coords(lat=("index", dsgr_red.lat))

<IPython.core.display.Javascript object>

In [13]:
# populate the geometry coordinate with metadata
add_geom_attrs = {
    "geometry": {
        "long_name": "World administrative boundaries (polygons)",
        "geometry_type": "polygon",
        "units": "degree",
        "comment": "These world boundaries (2020 version) are available from the World Bank, distributed in well-known binary format (wkb)",
        "crs_wkt": f"{glob_regions.crs.to_epsg()}",
    },
}

for k, v in add_geom_attrs.items():
    ds[k].attrs = add_geom_attrs[k]

<IPython.core.display.Javascript object>

### Make CF compliant alterations to the NetCDF files (dataset dependent)

In [14]:
# open datasets
# ds = xr.open_dataset(dataset_dir_path)

# check original dataset
ds

<xarray.Dataset>
Dimensions:                                           (index: 108)
Coordinates:
  * index                                             (index) int64 0 1 ... 109
    geometry                                          (index) object b'\x01\x...
    lon                                               (index) float64 104.0 ....
    lat                                               (index) float64 36.52 ....
Data variables:
    Country                                           (index) object "People'...
    Potential exposed population in 2010 (Million)    (index) float64 335.9 ....
    Exposed GDP (EGDP) (Billion US$)                  (index) float64 1.532e+...
    Potential global subsidence index in 2010 (PGSI)  (index) float64 60.79 ....
    Potential exposed population in 2040 (Million)    (index) float64 345.0 ....
    Exposed GDP (EGDP) (Billion US$) in 2040          (index) float64 1.573e+...
    Potential subsidence index 2040                   (index) float64 51.25 ....

<IPython.core.display.Javascript object>

In [15]:
# NetCDF attribute alterations by means of metadata template
f_global = open(dataset_dir.joinpath("metadata_subsidence.json"))
meta_global = json.load(f_global)

for attr_name, attr_val in meta_global.items():
    if attr_name == 'PROVIDERS':
        attr_val = json.dumps(attr_val)
    ds.attrs[attr_name] = attr_val

ds.attrs['Conventions'] = "CF-1.8"

<IPython.core.display.Javascript object>

In [16]:
dsn = copy.deepcopy(ds)

# rename dims and vars
dsn = dsn.rename_dims({"index": "stations"})
dsn = dsn.rename_vars({"Country": "country"})

# move variable to coords to avoid duplication in dimensions
#dsn = dsn.set_coords(["country"])
dsn = dsn.assign_coords({"country": ("stations", np.array(dsn["country"].values, dtype="S"))})

# merge variables for 2010 and 2040 into one variable
data_array_eapa = np.concatenate([dsn["Potential exposed population in 2010 (Million)"].values, dsn["Potential exposed population in 2040 (Million)"].values])
data_array_eapar = data_array_eapa.reshape((2,len(dsn["Potential exposed population in 2010 (Million)"].values)))
data_array_egdp = np.concatenate([dsn["Exposed GDP (EGDP) (Billion US$)"].values, dsn["Exposed GDP (EGDP) (Billion US$) in 2040"].values])
data_array_egdpr = data_array_egdp.reshape((2,len(dsn["Exposed GDP (EGDP) (Billion US$)"].values)))
data_array_epsi = np.concatenate([dsn["Potential global subsidence index in 2010 (PGSI)"].values, dsn["Potential subsidence index 2040"].values])
data_array_epsir = data_array_epsi.reshape((2,len(dsn["Potential global subsidence index in 2010 (PGSI)"].values)))

# remove variables
dsn = dsn.drop(["index", "Potential exposed population in 2010 (Million)", "Potential exposed population in 2040 (Million)", "Exposed GDP (EGDP) (Billion US$)", 
            "Exposed GDP (EGDP) (Billion US$) in 2040", "Potential global subsidence index in 2010 (PGSI)", "Potential subsidence index 2040"])

# add dimensions
dsn = dsn.expand_dims(dim={"time": [2010, 2040]})

# add variables
dsn = dsn.assign(eapa=(["time", "stations"], data_array_eapar))
dsn = dsn.assign(egdp=(["time", "stations"], data_array_egdpr))
dsn = dsn.assign(epsi=(["time", "stations"], data_array_epsir))

# add or change certain variable / coordinate attributes
dataset_attributes = {
    "time": {"long_name": "Time", "units": "yr"}, # TODO: year as unit raises a warning for not being exactly a calender year
    "lon": {"standard_name": "longitude", "long_name": "Longitude", "units": "degrees_east"},
    "lat": {"standard_name": "longitude", "long_name": "Latitude", "units": "degrees_north"},
    "country": {"long_name": "Country", "units": "1"},
    "eapa": {"long_name": "Expected annual people affected", "units": "1e6"},
    "egdp": {"long_name": "Exposed GDP (in US$)", "units": "1e9"}, # TODO: check how unit works in the F/E, compare to CFR CoCliCo dataset
    "epsi": {"long_name": "Potential subsidence index", "units": "1"}, # set to 1 if no unit
}  # specify custom (CF convention) attributes

# add / overwrite attributes
for k, v in dataset_attributes.items():
    try:
        dsn[k].attrs = dataset_attributes[k]
    except:
        continue

<IPython.core.display.Javascript object>

In [17]:
# check the xarray dataset, best practice is to have as many as possible bold dimensions (dimension == coordinate name).
# in this way, the Front-End can access the variable directly without having to index the variable first

dsn

<xarray.Dataset>
Dimensions:   (time: 2, stations: 108)
Coordinates:
  * time      (time) int32 2010 2040
    country   (stations) |S26 b"People's Republic of China" ... b'Zambia'
    geometry  (stations) object b'\x01\x03\x00\x00\x00\x01\x00\x00\x00A\x00\x...
    lon       (stations) float64 104.0 -112.5 138.0 29.86 ... 20.79 45.85 27.77
    lat       (stations) float64 36.52 45.69 37.55 26.49 ... 44.22 6.063 -13.46
Dimensions without coordinates: stations
Data variables:
    eapa      (time, stations) float64 335.9 39.31 27.22 ... 0.2337 0.002928
    egdp      (time, stations) float64 1.532e+03 1.902e+03 ... 0.0 0.004284
    epsi      (time, stations) float64 60.79 54.23 40.96 40.52 ... 0.0 0.0 0.0
Attributes: (12/21)
    TITLE:               Land Subsidence Threat
    TITLE_ABBREVIATION:  sub_threat
    DESCRIPTION:         Subsidence, the lowering of Earth's land surface, is...
    SHORT_DESCRIPTION:   To raise awareness and inform decision-making, we ev...
    INSTITUTION:         Instituto Geológico y Minero de España
    PROVIDERS:           {"name": "Instituto Geol\u00f3gico y Minero de Espa\...
    ...                  ...
    DOI:                 https://www.science.org/doi/10.1126/science.abb8549
    LONG_NAME:           sub_threat
    UNITS:               
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [18]:
# TODO: make CF compliant...
# TODO: how do nuts regions work.. (see notes..)
# NUTS2 regions
# COUNTRY code names..? check SM data
# differenct variables with different units and dimensions?
# how to make a multidimensional xarray file?

<IPython.core.display.Javascript object>

In [19]:
# write to NetCDF file to check compliancy

# prevent file locking, see: https://github.com/pydata/xarray/issues/2376
import os
os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'

dsn.to_netcdf(path=dataset_dir.joinpath(dataset_out_file + "_CF.nc"))

<IPython.core.display.Javascript object>

### Check CF compliancy altered NetCDF files

In [20]:
%%capture cap --no-stderr
# check altered CF compliancy

check_compliancy(testfile=dataset_dir.joinpath(dataset_out_file + "_CF.nc"), working_dir=CF_dir)

<IPython.core.display.Javascript object>

In [21]:
# save altered CF compliancy
save_compliancy(
    cap,
    testfile=dataset_dir.joinpath(dataset_out_file + "_CF.nc"),
    working_dir=CF_dir,
)

{'P:\\11208003-latedeo2022\\020_InternationalDeltaPortfolio\\datasets\\00_mapping_global_threat_of_land_subsidence\\Global_TLS_CF.nc': {'warnings': '0', 'errors': '0'}}


<IPython.core.display.Javascript object>

### Write data to Zarr files

In [22]:
# export to zarr in write mode (to overwrite if exists)
dsn.to_zarr(dataset_dir.joinpath("%s.zarr" % dataset_out_file), mode="w")

<IPython.core.display.Javascript object>